In [2]:
import numpy as np
import os
from os.path import isfile
from PIL import Image as Img
from datetime import datetime

import keras
from keras.models import Sequential, Model
from keras.utils import plot_model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, Flatten, MaxPooling2D, Reshape
from keras.layers import Conv2D, BatchNormalization, Lambda, Permute, GRU
from keras.layers.advanced_activations import ELU
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import backend
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop
from keras import regularizers

import librosa
import librosa.display
import matplotlib.pyplot as plt

import tensorflow as tf

In [3]:
SHUFFLE_BUFFER = 100
BATCH_SIZE = 32
NUM_CLASSES = 12

# Create a description of the features.  
feature_description = {
    'feature0': tf.FixedLenFeature([32768], tf.float32),
    'feature1': tf.FixedLenFeature([1], tf.int64)
}

def _parse_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
    parsed_example = tf.parse_single_example(example_proto, feature_description)
    parsed_example["feature0"] = tf.transpose(tf.reshape(parsed_example['feature0'], (256,128)))
    return parsed_example

def create_dataset(filepath):
    
    dataset = tf.data.TFRecordDataset(filepath)
    
    dataset = dataset.map(_parse_function) #, num_parallel_calls=8)
    
    # This dataset will go on forever
    dataset = dataset.repeat()
    
    # Set the number of datapoints you want to load and shuffle 
    dataset = dataset.shuffle(SHUFFLE_BUFFER)
    dataset = dataset.batch(BATCH_SIZE)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create your tf representation of the iterator
    feature = iterator.get_next()
    #print(feature)
    lmfcc = feature["feature0"]
    label = feature["feature1"]
    
    # Bring your picture back in shape
    lmfcc = tf.reshape(lmfcc, [-1,128, 256])
    
    # Create a one hot array for your labels
    label = tf.one_hot(label, NUM_CLASSES)
    print("inp",lmfcc.shape)
    print("inp",label.shape)

    return lmfcc, label

In [4]:
lmfcc, label = create_dataset("../data/debug/sample.tfrecords")

inp (?, 128, 256)
inp (?, 1, 12)


In [5]:
def get_callbacks(checkpoint_name):
    logDir = "./Graph/" + datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
    tb = TensorBoard(log_dir=logDir,
                     histogram_freq=2,
                     write_graph=True,
                     write_images=True,
                     write_grads=True,
                     update_freq='epoch')

#     tb_callback = TensorBoard(
#         log_dir='../models/logs/',
#         histogram_freq=1,
#         batch_size=32,
#         write_graph=True,
#         write_grads=False,
#         write_images=False,
#         embeddings_freq=0,
#         embeddings_layer_names=None,
#         embeddings_metadata=None,
#     )

    checkpoint_callback = ModelCheckpoint('../models/' + checkpoint_name +
                                          '{epoch:02d}-{val_loss:.2f}.hdf5',
                                          monitor='val_acc',
                                          verbose=1,
                                          save_best_only=True,
                                          mode='max')

    reducelr_callback = ReduceLROnPlateau(monitor='val_acc',
                                          factor=0.5,
                                          patience=10,
                                          min_delta=0.01,
                                          verbose=1)

    callback_list = [checkpoint_callback, reducelr_callback]

    return callback_list

In [10]:
# Data iterator
lmfcc, label = create_dataset("../data/debug/train.tfrecords")
lmfcc_val, label_val = create_dataset("../data/debug/val.tfrecords")

#Build network
NUM_CLASSES = 12  # Must Change in the tf reader as well
N_LAYERS = 3
CONV_FILTER_COUNT = 64
FILTER_LENGTH = 5

POOL_SIZE = 2

GRU_COUNT = 64
NUM_HIDDEN = 128
L2_regularization = 0.001

# Input
model_input = keras.layers.Input(tensor=lmfcc)
print(model_input.shape)
layer = Permute((2, 1), input_shape=(128, 256))(model_input)
print(layer.shape)
# resize_shape = model.output_shape[2] * model.output_shape[3]
# model.add(Reshape((model.output_shape[1], resize_shape)))

# model.add(Permute((2, 1), input_shape=(10, 64)))
# now: model.output_shape == (None, 64, 10)

# Conv1D , input_shape=(10, 128) for time series sequences of 10 time steps with 128 features per step
# 1st conv
layer = Conv1D(filters=CONV_FILTER_COUNT,
               kernel_size=FILTER_LENGTH)(layer)  #(model_input)
layer = Activation('relu')(layer)
layer = MaxPooling1D(pool_size=POOL_SIZE, strides=POOL_SIZE)(layer)
layer = Dropout(0.2)(layer)

for i in range(N_LAYERS - 1):
    layer = Conv1D(filters=128, kernel_size=FILTER_LENGTH)(layer)
    layer = Activation('relu')(layer)
    layer = MaxPooling1D(pool_size=POOL_SIZE, strides=POOL_SIZE)(layer)
    layer = Dropout(0.4)(layer)

## LSTM Layer
layer = GRU(GRU_COUNT, return_sequences=True)(layer)
layer = GRU(GRU_COUNT, return_sequences=False)(layer)

layer = Dropout(0.4)(layer)

## Softmax Output
layer = Dense(NUM_CLASSES)(layer)
layer = Activation('softmax')(layer)
model_output = layer

#model_output = Dense(NUM_CLASSES, activation='relu')(model_output)

#Create your model
train_model = Model(inputs=model_input, outputs=model_output)

#compile
train_model.compile(loss='categorical_crossentropy',
                    optimizer=Adam(lr=0.01),
                    metrics=['accuracy'],
                    target_tensors=[label])

#Train the model
#steps per epoch could be viewed as dataset/batchsize
batch_size = 16
# Better to change checkpoint name before run
train_model.fit(epochs=70,
                steps_per_epoch=10,
                validation_data=(lmfcc_val, label_val),
                validation_steps=10,
                callbacks=get_callbacks(checkpoint_name="trail"))

inp (?, 128, 256)
inp (?, 1, 12)
inp (?, 128, 256)
inp (?, 1, 12)
(?, 128, 256)
(?, 256, 128)
Epoch 1/70
10/10 [==============================] - 4s 403ms/step - loss: 2.5762 - acc: 0.2671 - val_loss: 4.5677 - val_acc: 0.0469

Epoch 00001: val_acc improved from -inf to 0.04688, saving model to ../models/trail01-4.57.hdf5
Epoch 2/70
10/10 [==============================] - 2s 157ms/step - loss: 3.6402 - acc: 0.1194 - val_loss: 3.3797 - val_acc: 0.0094

Epoch 00002: val_acc did not improve from 0.04688
Epoch 3/70
10/10 [==============================] - 2s 159ms/step - loss: 3.0235 - acc: 0.1496 - val_loss: 3.7436 - val_acc: 0.0156

Epoch 00003: val_acc did not improve from 0.04688
Epoch 4/70
10/10 [==============================] - 2s 187ms/step - loss: 2.5453 - acc: 0.1884 - val_loss: 3.1613 - val_acc: 0.0500

Epoch 00004: val_acc improved from 0.04688 to 0.05000, saving model to ../models/trail04-3.16.hdf5
Epoch 5/70
10/10 [==============================] - 2s 194ms/step - loss: 2.544

10/10 [==============================] - 2s 158ms/step - loss: 2.2378 - acc: 0.1031 - val_loss: 2.2532 - val_acc: 0.3406

Epoch 00042: val_acc improved from 0.32500 to 0.34063, saving model to ../models/trail42-2.25.hdf5
Epoch 43/70
10/10 [==============================] - 2s 156ms/step - loss: 2.6005 - acc: 0.0964 - val_loss: 2.0305 - val_acc: 0.3000

Epoch 00043: val_acc did not improve from 0.34063
Epoch 44/70
10/10 [==============================] - 2s 184ms/step - loss: 2.7112 - acc: 0.0915 - val_loss: 1.9967 - val_acc: 0.0969

Epoch 00044: val_acc did not improve from 0.34063
Epoch 45/70
10/10 [==============================] - 2s 192ms/step - loss: 2.6705 - acc: 0.0623 - val_loss: 2.1805 - val_acc: 0.2938

Epoch 00045: val_acc did not improve from 0.34063
Epoch 46/70
10/10 [==============================] - 2s 171ms/step - loss: 2.3441 - acc: 0.1247 - val_loss: 2.4575 - val_acc: 0.1750

Epoch 00046: val_acc did not improve from 0.34063
Epoch 47/70
10/10 [========================

KeyboardInterrupt: 

In [11]:
train_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 128, 256)          0         
_________________________________________________________________
permute_4 (Permute)          (None, 256, 128)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 252, 64)           41024     
_________________________________________________________________
activation_10 (Activation)   (None, 252, 64)           0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 126, 64)           0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 126, 64)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 122, 128)          41088     
__________

In [8]:
#RNN
# Data iterator
lmfcc, label = create_dataset("../data/debug/train.tfrecords")
lmfcc_val, label_val = create_dataset("../data/debug/val.tfrecords")

#Build network
NUM_CLASSES = 12  # Must Change in the tf reader as well
N_LAYERS = 3
CONV_FILTER_COUNT = 64
FILTER_LENGTH = 5

POOL_SIZE = 2

GRU_COUNT = 64
NUM_HIDDEN = 128
L2_regularization = 0.001

# Input
model_input = keras.layers.Input(tensor=lmfcc)
print(model_input.shape)
layer = Permute((2, 1), input_shape=(128, 256))(model_input)
print(layer.shape)
# resize_shape = model.output_shape[2] * model.output_shape[3]
# model.add(Reshape((model.output_shape[1], resize_shape)))

# model.add(Permute((2, 1), input_shape=(10, 64)))
# now: model.output_shape == (None, 64, 10)



## LSTM Layer
layer = LSTM(GRU_COUNT, return_sequences=True)(layer)
layer = LSTM(GRU_COUNT, return_sequences=False)(layer)

## Softmax Output
layer = Dense(NUM_CLASSES)(layer)
layer = Activation('softmax')(layer)
model_output = layer

#model_output = Dense(NUM_CLASSES, activation='relu')(model_output)

#Create your model
train_model = Model(inputs=model_input, outputs=model_output)

#compile
train_model.compile(loss='categorical_crossentropy',
                    optimizer=Adam(lr=0.01),
                    metrics=['accuracy'],
                    target_tensors=[label])

#Train the model
#steps per epoch could be viewed as dataset/batchsize
batch_size = 16
# Better to change checkpoint name before run
train_model.fit(epochs=70,
                steps_per_epoch=10,
                validation_data=(lmfcc_val, label_val),
                validation_steps=10,
                callbacks=get_callbacks(checkpoint_name="trail"))

inp (?, 128, 256)
inp (?, 1, 12)
inp (?, 128, 256)
inp (?, 1, 12)
(?, 128, 256)
(?, 256, 128)
Epoch 1/70
10/10 [==============================] - 6s 584ms/step - loss: 2.4703 - acc: 0.2848 - val_loss: 3.7346 - val_acc: 0.0531

Epoch 00001: val_acc improved from -inf to 0.05312, saving model to ../models/trail01-3.73.hdf5
Epoch 2/70
10/10 [==============================] - 4s 357ms/step - loss: 2.8841 - acc: 0.0875 - val_loss: 2.4565 - val_acc: 0.0281

Epoch 00002: val_acc did not improve from 0.05312
Epoch 3/70
10/10 [==============================] - 4s 377ms/step - loss: 2.4153 - acc: 0.0625 - val_loss: 2.8473 - val_acc: 0.0125

Epoch 00003: val_acc did not improve from 0.05312
Epoch 4/70
10/10 [==============================] - 4s 363ms/step - loss: 2.1811 - acc: 0.3719 - val_loss: 3.0695 - val_acc: 0.0594

Epoch 00004: val_acc improved from 0.05312 to 0.05937, saving model to ../models/trail04-3.07.hdf5
Epoch 5/70
10/10 [==============================] - 4s 366ms/step - loss: 2.196


Epoch 00042: val_acc did not improve from 0.39062

Epoch 00042: ReduceLROnPlateau reducing learning rate to 0.0012499999720603228.
Epoch 43/70
10/10 [==============================] - 4s 447ms/step - loss: 2.3580 - acc: 0.0219 - val_loss: 2.0827 - val_acc: 0.3688

Epoch 00043: val_acc did not improve from 0.39062
Epoch 44/70
10/10 [==============================] - 4s 410ms/step - loss: 2.4920 - acc: 0.0656 - val_loss: 1.9629 - val_acc: 0.2281

Epoch 00044: val_acc did not improve from 0.39062
Epoch 45/70
10/10 [==============================] - 5s 471ms/step - loss: 2.5237 - acc: 0.2000 - val_loss: 2.1471 - val_acc: 0.0187

Epoch 00045: val_acc did not improve from 0.39062
Epoch 46/70
10/10 [==============================] - 4s 366ms/step - loss: 2.2128 - acc: 0.3937 - val_loss: 2.4591 - val_acc: 0.0250

Epoch 00046: val_acc did not improve from 0.39062
Epoch 47/70
10/10 [==============================] - 4s 448ms/step - loss: 1.9788 - acc: 0.3531 - val_loss: 2.5509 - val_acc: 0.0750